In [1]:
import sys
sys.path.append('../')
from megaparse.cdp.utils.diff_agent import DifferenceAgent, ContextType
from megaparse.cdp.utils.query_engine import DiffQueryEngine
from pathlib import Path
from megaparse.cdp.utils.question_generator import QuestionGenerator

In [2]:
diff_query_engine = DiffQueryEngine(Path("../megaparse/cdp/CDP_QUAL_CHART_01_CHARTE_PRODUITS.md"), top_k= 15)
question_generator = QuestionGenerator()

In [3]:
agent = DifferenceAgent(diff_query_engine, document_context= None, question_generator=question_generator)

In [4]:
questions = agent.generate_questions(Path("../megaparse/cdp/exemple_fournisseur.xlsx"), tab_name = 'Fiche recette', language_verification= True)
print(f"{len(questions)} questions générées") #type: ignore

Generating Questions ...
Verifying language and translating questions ...
37 questions générées)


In [ ]:
additional_contexts = [
                        ContextType(category='Coeur de Gamme', context='pour les produits de la catégorie Coeur de Gamme ?'),
                        ]
generated_df = agent.run(additional_context=additional_contexts[0])


In [24]:
import pandas as pd
def generate_authorisation_report(df, output_file):
    # Ensure the dataframe has the required columns
    if 'name' not in df.columns or 'decision' not in df.columns:
        raise ValueError("DataFrame must contain 'name' and 'decision' columns")

    # Extract forbidden and to avoid ingredients
    forbidden_ingredients = df[df['decision'] == 'Forbidden'][['name', 'detailed_answer']].values.tolist()
    to_avoid_ingredients = df[df['decision'] == 'To Avoid'][['name', 'detailed_answer']].values.tolist()

    # Create the markdown content
    markdown_content = "# Coup de Pates\n\n"
    markdown_content += "## Rapport d'analyse\n\n"

    if forbidden_ingredients:
        markdown_content += "### Ingrédients utilisés **Interdits**\n\n"
        for ingredient, detail in forbidden_ingredients:
            markdown_content += f"**{ingredient}**: *{detail}*\n\n"
    else:
        markdown_content += "### Ingrédients utilisés **Interdits**\n- Aucun\n\n"

    markdown_content += "\n"

    if to_avoid_ingredients:
        markdown_content += "### Ingrédients utilisés **à éviter**\n"
        for ingredient, detail in to_avoid_ingredients:
            markdown_content += f"**{ingredient}** : *{detail}*\n\n"
    else:
        markdown_content += "### To Avoid Ingredients\n- None\n\n"

    # Write the markdown content to the file
    with open(output_file, 'w') as file:
        file.write(markdown_content)


generate_authorisation_report(generated_df, 'authorization_report.md')